# HHL Algorithm

**The HHL Algorthim is a quantum algorithm used to solve linear systems of equations that relies on quantum phase estimation**

Rescaling system to quantum states $|b\rangle$ and $|x\rangle$:

For A Hermitian:

$$A|x\rangle = |b\rangle$$

Using spectral decomposition:
$$A = \sum _{j=0}^{N-1}\lambda_j |u_j\rangle\langle u_j|$$

$$A^{-1} = \sum_{j=0}^{N-1}\lambda_j |u_j\rangle \langle u_j|$$

Then, writing $|b\rangle$ in the basis of $A$:
$$|b\rangle = \sum_{j=0}^{N-1} b_j |u_j\rangle$$



Example:
$$A = \left(\begin{matrix} 1 & -1/3 \\ -1/3 & 1 \end{matrix} \right)$$
$$|b\rangle =\left( \begin{matrix} 1 \\ 0\end{matrix}\right)$$

Calculate eigenvalues to determine $t$ for binary representation. HHL does not require knowing eigenvalues.


In [1]:
import numpy as np

In [2]:
A = np.matrix([[1,-1/3],[-1/3,1]])
print(A)
eigvals = np.linalg.eigvals(A)
print(eigvals)

[[ 1.         -0.33333333]
 [-0.33333333  1.        ]]
[1.33333333 0.66666667]


$$\lambda_1 = 2/3,\lambda_2 = 4/3$$

QPE output $n_l$-bit binary approximation to $\frac{\lambda_j t}{2\pi}$

If chose $t = 2\pi \frac{3}{8}$, QPE gives 2bit binary approximation: 1/4 and 1/2
Represented as:
$$|01\rangle_{n_l}, |10\rangle_{n_l}$$

$$|b\rangle_{n_b} = \sum_{j=1}^2 \frac{1}{\sqrt{2}}|u_j\rangle_{n_b}$$

## Algorithim Implementation

**State preparation**

$$|b\rangle = |0\rangle$$

**Applying QPE**

$$\frac{1}{\sqrt{2}}|01\rangle |u_1\rangle+\frac{1}{\sqrt{2}}|10\rangle |u_2\rangle$$

**Conditioned Rotation C = 3/8**
$$\frac{1}{\sqrt{2}}|01\rangle |u_1\rangle \left(\sqrt{1-\frac{(3/8)^2}{(1/4)^2}}|0\rangle +\frac{3/8}{1/4}|1\rangle \right) +\frac{1}{\sqrt{2}}|10\rangle |u_2\rangle\left(\sqrt{1-\frac{(3/8)^2}{(1/2)^2}}|0\rangle +\frac{3/8}{1/2}|1\rangle \right) $$

$$= \frac{1}{\sqrt{2}} |01\rangle|u_1\rangle\left(\sqrt{1-\frac{9}{4}}|0\rangle+\frac{3}{2}|1\rangle \right)+ \frac{1}{\sqrt{2}} |10\rangle|u_2\rangle\left(\sqrt{1-\frac{9}{16}}|0\rangle+\frac{3}{4}|1\rangle \right)$$
**Applying QPE$^\dagger$**
$$ \frac{1}{\sqrt{2}} |00\rangle|u_1\rangle\left(\sqrt{1-\frac{9}{4}}|0\rangle+\frac{3}{2}|1\rangle \right)+ \frac{1}{\sqrt{2}} |00\rangle|u_2\rangle\left(\sqrt{1-\frac{9}{16}}|0\rangle+\frac{3}{4}|1\rangle \right)$$

**State for auxiliary qb to be $|1\rangle$**
$$\frac{1}{\sqrt{45/32}}\left( \frac{3}{2\sqrt{2}}|u_1\rangle + \frac{3}{4\sqrt{2}}|u_2\rangle\right) = \frac{|x\rangle}{||x||}$$

**Probability of measuring 1**
$$P(1) = ||x||^2 = \frac{45}{32}$$

## Qiskit Implementation

In [3]:
from qiskit import Aer, transpile, assemble
from qiskit.circuit.library import QFT
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.quantum_info import state_fidelity
from qiskit.aqua.algorithms import HHL, NumPyLSsolver
from qiskit.aqua.components.eigs import EigsQPE
from qiskit.aqua.components.reciprocals import LookupRotation
from qiskit.aqua.operators import MatrixOperator
from qiskit.aqua.components.initial_states import Custom

In [4]:
 # Eigenvector creation

def Eigenvals(A,n_aux, n_time, neg_eigvals):
    qfts = [None,None]
    if neg_eigvals:
        n_aux += 1
        qfts = [QFT(n_aux -1), QFT(n_aux-1).inverse()]
        
    return EigsQPE(MatrixOperator(matrix=A), #QPE function call, operator A
                  QFT(n_aux).inverse(), #inverse QFT circuit
                  num_time_slices = n_time,
                  num_ancillae = n_aux, #number of ancillae = number of auxillary qb
                  expansion_mode = 'suzuki', #trotter or suzuki
                  expansion_order = 2,
                  evo_time = None, #this is t for QPE
                  negative_evals = neg_eigvals,
                  ne_qfts = qfts)
        

In [5]:
#function to calculate fidelity of solution

def fidelity(hhl,ref):
    #normalize hhl and ref
    sol_hhl_norm = hhl/np.linalg.norm(hhl)
    print(sol_hhl_norm)
    sol_ref_norm = ref/np.linalg.norm(ref)
    print(sol_ref_norm)
    f = state_fidelity(sol_hhl_norm,sol_ref_norm)
    print("Fidelity:  %f " %f)

In [6]:
#initialize operator A and vector state b

A = [[1,-1/3],[-1/3,1]]
print(A)
b = [1,0]

[[1, -0.3333333333333333], [-0.3333333333333333, 1]]


In [7]:
init_size = len(b)

A, b, truncate_powerdim, truncate_hermitian = HHL.matrix_resize(A, b)
print(A)

#initialize eigenvalue finder

eigs = Eigenvals(A, 3, 50, False)
num_q , num_a = eigs.get_register_sizes()

#initialize state

init_state = Custom(num_q, state_vector = b)

#reciprocal rotation
reciprocal = LookupRotation(negative_evals=eigs._negative_evals, evo_time=eigs._evo_time)

algorithm = HHL(A, b, truncate_powerdim, truncate_hermitian, eigs,
           init_state, reciprocal, num_q, num_a, init_size)

[[ 1.         -0.33333333]
 [-0.33333333  1.        ]]


In [8]:
#approximated solution using HHL
result = algorithm.run(QuantumInstance(Aer.get_backend('statevector_simulator')))
print("Solution:", np.round(result['solution'], 5))


Solution: [1.13586-0.j 0.40896-0.j]


In [9]:
#actual solution
result_ref = NumPyLSsolver(A, b).run()
print("Classical Solution:", np.round(result_ref['solution'], 5))


Classical Solution: [1.125 0.375]


In [10]:
#probability of measuring 1
print("Probability: %f" % result['probability_result'])


Probability: 0.056291


In [11]:
fidelity(result['solution'], result_ref['solution'])

[0.94087437-1.21798945e-12j 0.33875568-1.45628611e-13j]
[0.9486833  0.31622777]
Fidelity:  0.999432 


## Optimized HHL

Can be used for any problem with
$$A = \left(\begin{matrix}a& b \\ b & a \end{matrix}\right) $$
and 
$$|b\rangle = \left(\begin{matrix} \cos(\theta)  \\ \sin(\theta) \end{matrix}\right)$$

In [12]:
from qiskit import QuantumRegister, QuantumCircuit

In [13]:
t  = np.pi *3/4 #t calculated from eigenvalues

nqb = 4 #number of qb

nb = 1 #number of qb representing solution
nl = 2 #number of qb representing eigenvalues

theta =  0 #angle to define |b>

#for example above:
a = 1 
b = -1/3 

#initialize registers
qr = QuantumRegister(nqb)


#create quantum circuit
qc = QuantumCircuit(qr)
qrb = qr[0:nb]
qrl = qr[nb:nb+nl]
qra = qr[nb+nl:nb+nl+1]

#prepare the state
qc.ry(2*theta,qrb[0])

#QPE e^{iAt}
for qu in qrl:
    qc.h(qu)
    
#qc.p is a phase gate
    
qc.p(a*t, qrl[0])
qc.p(a*t*2,qrl[0])
qc.u(b*t, - np.pi/2, np.pi/2,qrb[0])

#controlled e^{iAt} on first eigenvalue
param = b*t

qc.p(np.pi/2,qrb[0])
qc.cx(qrl[0],qrb[0])
qc.ry(param,qrb[0])
qc.cx(qrl[0],qrb[0])
qc.ry(-param,qrb[0])
qc.p(3*np.pi/2,qrb[0])

     
#Inverse QFT
qc.h(qrl[1])
qc.rz(-np.pi/4,qrl[1])
qc.cx(qrl[0],qrl[1])
qc.rz(np.pi/4,qrl[1])
qc.cx(qrl[0],qrl[1])
qc.rz(-np.pi/4,qrl[0])
qc.h(qrl[0])

# Eigenvalue rotation
t1=(-np.pi +np.pi/3 - 2*np.arcsin(1/3))/4
t2=(-np.pi -np.pi/3 + 2*np.arcsin(1/3))/4
t3=(np.pi -np.pi/3 - 2*np.arcsin(1/3))/4
t4=(np.pi +np.pi/3 + 2*np.arcsin(1/3))/4

qc.cx(qrl[1],qra[0])
qc.ry(t1,qra[0])
qc.cx(qrl[0],qra[0])
qc.ry(t2,qra[0])
qc.cx(qrl[1],qra[0])
qc.ry(t3,qra[0])
qc.cx(qrl[0],qra[0])
qc.ry(t4,qra[0])
qc.measure_all()


qc.draw(fold=-1)

┌───────┐┌──────────────────┐ ┌────────┐ ┌───┐┌──────────┐┌───┐┌─────────┐┌─────────┐                                                                                              ░ ┌─┐         
 q14_0: ┤ RY(0) ├┤ U(-π/4,-π/2,π/2) ├─┤ P(π/2) ├─┤ X ├┤ RY(-π/4) ├┤ X ├┤ RY(π/4) ├┤ P(3π/2) ├──────────────────────────────────────────────────────────────────────────────────────────────░─┤M├─────────
        └─┬───┬─┘└───┬─────────┬────┘┌┴────────┤ └─┬─┘└──────────┘└─┬─┘└─────────┘└─────────┘     ┌──────────┐     ┌───┐                                                                   ░ └╥┘┌─┐      
 q14_1: ──┤ H ├──────┤ P(3π/4) ├─────┤ P(3π/2) ├───■────────────────■───────■──────────────────■──┤ RZ(-π/4) ├─────┤ H ├────────■────────────────────────────────────────■─────────────────░──╫─┤M├──────
          ├───┤      └──┬───┬──┘     ├─────────┴┐                         ┌─┴─┐   ┌─────────┐┌─┴─┐└──────────┘     └───┘        │                                        │                 ░  ║ └╥┘┌─┐   
 q14_2: ──┤ H ├─────────┤ H ├────────┤ RZ(-π/4) ├─────────────────────────┤ X ├───┤ RZ(π/4) ├┤ X ├─────■────────────────────────┼────────────────────■───────────────────┼─────────────────░──╫──╫─┤M├───
          └───┘         └───┘        └──────────┘                         └───┘   └─────────┘└───┘   ┌─┴─┐    ┌──────────────┐┌─┴─┐┌──────────────┐┌─┴─┐┌─────────────┐┌─┴─┐┌────────────┐ ░  ║  ║ └╥┘┌─┐
 q14_3: ─────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────┤ RY(-0.69352) ├┤ X ├┤ RY(-0.87728) ├┤ X ├┤ RY(0.35368) ├┤ X ├┤ RY(1.2171) ├─░──╫──╫──╫─┤M├
                                                                                                     └───┘    └──────────────┘└───┘└──────────────┘└───┘└─────────────┘└───┘└────────────┘ ░  ║  ║  ║ └╥┘
meas: 4/══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╩══╩══╩══╩═
                                                                                                                                                                                              0  1  2  3

In [14]:
from qiskit import execute, BasicAer, ClassicalRegister, IBMQ
from qiskit.compiler import transpile
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter,  MeasurementFilter

In [25]:
provider = IBMQ.load_account()
backend = provider.get_backend('ibmqx2')

IBMQ.get_provider(hub='ibmqx2', group='open', project='main')


IBMQAccountCredentialsNotFound: 'No IBM Quantum Experience credentials found.'